In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import cv2
from PIL import Image

In [2]:
num_classes = 5

In [3]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Giriş boyutunu uygun şekilde ayarlayın
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
# Model yapısını oluştur
model = CNNModel(num_classes)  # Model yapısını aynı şekilde oluşturmalısınız

# Kaydedilmiş parametreleri yükle
model.load_state_dict(torch.load('soil.pth'))

# Modeli tahminlerde kullanabilirsiniz
model.eval()  # Modeli tahmin modunda etkinleştirin
# Burada tahminlerinizi yapabilirsiniz


CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)

In [4]:
image_path = 'blacksoil.jpg'  # Test görüntüsünün dosya yolu
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Görüntüyü BGR'den RGB'ye dönüştür
image_pil = Image.fromarray(image)

In [5]:
# Giriş görüntüsünü modele uygun hale getir
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = transform(image_pil)
input_image = input_image.unsqueeze(0)  # Batch boyutunu ekleyin (1 görüntü için)

In [8]:
# Veri yolu - Klasördeki verilerin kategori alt klasörlerde bulunduğunu varsayalım.
data_dir = 'Soil types/'
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [9]:
with torch.no_grad():
    output = model(input_image)

# Sınıflandırma sonuçlarını al
_, predicted = torch.max(output, 1)

# Sınıf tahminini bul
class_index = predicted.item()
class_label = dataset.classes[class_index]  # 'dataset' veri kümesinin sınıflarını içerdiğini varsayalım

print(f'Tahmin edilen sınıf: {class_label}')

Tahmin edilen sınıf: Black Soil
